#BERTopic on Quran Urdu Translations
In this notebook I have implemented Topic Modeling on Urdu translations based topic modelling technique BERTopic.

Shaista Zulfiqar

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install urduhack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB

In [ ]:
!pip install --upgrade keras tensorflow-addons


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score and train LDA. bertopic is used to train bertopic on our Quran-UTM dataset with using pretrained language models Multilingial MiniLM

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from urduhack.normalization import remove_diacritics
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame




In [ ]:

junapd = pd.read_table("", header=None, encoding='utf-8') #please provide your path

print(junapd.head(5))

stopspd=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])

stopspd

                                                   0
0  1|1|اللہ کے نام سےشروع جو نہایت مہربان ہمیشہ ر...
1  1|2|سب تعریفیں اللہ ہی کے لئے ہیں جو تمام جہان...
2            1|3|نہایت مہربان بہت رحم فرمانے والا ہے
3                            1|4|روزِ جزا کا مالک ہے
4  1|5|(اے اللہ!) ہم تیری ہی عبادت کرتے ہیں اور ہ...


,List
0,کی
1,ہیں
2,ہے
3,رہا
4,رہی
...,...
396,گئی
397,ہونے
398,وجہ
399,ہوگیا


## Cleaning of Data
After collecting the eight Quran Urdu translations, we observed these translation had some irrelevant information, such as metadata, punctuation, and diacritics. So we cleaned the translations for topic modeling

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling. Stopwords are removed in post preprocessing phase

In [ ]:
import re
#Remove Urdu Punctuation
def remove_urdu_punctuation(text):
    pattern = r'[؛؟،٫٬‘’“”«»!"٪&\'\*\+,-./:;<=>؟@^_`()[]{|}~]'

    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text


In [ ]:
# Remove trailing metadata lines
index_to_drop = junapd[junapd[0].str.startswith("#")].index
junapd.drop(index_to_drop, inplace=True)

# Remove verse reference from every line
junapd[0] = junapd[0].str.replace(r'\d+\|\d+\|', '', regex=True)

# Remove diacritics
junapd[0] = junapd[0].apply(remove_diacritics)

# Remove punctuation
junapd[0] = junapd[0].apply(remove_urdu_punctuation)

# Reset index
junapd.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(junapd.head())



Cleaned DataFrame:
                                                   0
0  اللہ کے نام سےشروع جو نہایت مہربان ہمیشہ رحم ف...
1  سب تعریفیں اللہ ہی کے لئے ہیں جو تمام جہانوں ک...
2                نہایت مہربان بہت رحم فرمانے والا ہے
3                                 روز جزا کا مالک ہے
4  (اے اللہ!) ہم تیری ہی عبادت کرتے ہیں اور ہم تج...


In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

junapd[0] = junapd[0].apply(remove_nonbreaking_space)

In [ ]:
# Save the cleaned DataFrame to a text file
file_path = 'cleaned_Qadri_text.txt'
junapd[0].to_csv(file_path, sep='\n', index=False, header=False, encoding='utf-8')

print(f'Text has been saved to {file_path}')

Text has been saved to cleaned_Qadri_text.txt


In [ ]:
# Convert the DataFrame to a list of strings
data = junapd[0].tolist()

In [ ]:
print(len(data))

6236


In [ ]:
print(data[56:90])

['اور جب ہم نے تمہیں (بچانے کے) لئے دریا کو پھاڑ دیا سو ہم نے تمہیں (اس طرح) نجات عطا کی اور (دوسری طرف) ہم نے تمہاری آنکھوں کے سامنے قوم فرعون کو غرق کر دیا،', 'اور (وہ وقت بھی یاد کرو) جب ہم نے موسی (علیہ السلام) سے چالیس راتوں کا وعدہ فرمایا تھا پھر تم نے موسی (علیہ السلام کے چلّہء اعتکاف میں جانے) کے بعد بچھڑے کو (اپنا) معبود بنا لیا اور تم واقعی بڑے ظالم تھے،', 'پھر ہم نے اس کے بعد (بھی) تمہیں معاف کر دیا تاکہ تم شکرگزار ہو جاؤ،', 'اور جب ہم نے موسی (علیہ السلام) کو کتاب اور حق و باطل میں فرق کرنے والا (معجزہ) عطا کیا تاکہ تم راہ ہدایت پاؤ،', 'اور جب موسی (علیہ السلام) نے اپنی قوم سے کہا: اے میری قوم! بیشک تم نے بچھڑے کو (اپنا معبود) بنا کر اپنی جانوں پر (بڑا) ظلم کیا ہے، تو اب اپنے پیدا فرمانے والے (حقیقی رب) کے حضور توبہ کرو، پس (آپس میں) ایک دوسرے کو قتل کر ڈالو (اس طرح کہ جنہوں نے بچھڑے کی پرستش نہیں کی اور اپنے دین پر قائم رہے ہیں وہ بچھڑے کی پرستش کر کے دین سے پھر جانے والوں کو سزا کے طور پر قتل کر دیں)، یہی (عمل) تمہارے لئے تمہارے خالق کے نزدیک بہترین (توبہ) ہے، پھر اس نے ت

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". We take MiniLM Model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(data, show_progress_bar=True)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

[[ 0.01926055  0.6384604  -0.27696165 ... -0.13326076  0.36460337
   0.01672293]
 [-0.03112397  0.6492219  -0.22533192 ... -0.10756381  0.04437538
   0.06413698]
 [ 0.03558538  0.23179418 -0.128763   ... -0.12293386  0.37514505
   0.01303583]
 ...
 [ 0.00775517  0.39919353 -0.14593394 ... -0.10231502  0.23667261
   0.02902103]
 [-0.00548704  0.02401588 -0.15880702 ...  0.01827102  0.15145205
   0.05712897]
 [-0.00503411  0.15777272 -0.21931344 ... -0.154871    0.2978512
  -0.11685145]]


In [ ]:
stop_words_list = stopspd['List'].tolist()

In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stop_words_list)


In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_components=4, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True,  top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model, verbose=True, vectorizer_model=vectorizer_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(data,embeddings)

2024-06-09 09:29:18,867 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-09 09:30:06,839 - BERTopic - Dimensionality - Completed ✓
2024-06-09 09:30:06,842 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-09 09:30:06,983 - BERTopic - Cluster - Completed ✓
2024-06-09 09:30:07,004 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-09 09:30:07,224 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[3, 3, 1, 0, 1, 0, 1, 3, 1, 2, 2, 2, 2, 3, 2, 3, 3, 0, 0, 2, 3, 3, 0, 3, 0, 3, 3, 1, 1, 3, 0, 2, 3, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 2, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2, 3, 0, 0, 1, 1, 1, 1, 1, 3, 1, 0, 3, 3, 3, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 1, 2, 2, 3, 2, 3, 3, 2, 2, 3, 1, 2, 3, 3, 1, 3, 3, 2, 3, 2, 2, 3, 3, 3, 1, 1, 3, 2, 3, 2, 0, 2, 3, 3, 3, 3, 1, 1, 3, 1, 3, 1, 0, 3, 3, 3, 1, 2, 3, 0, 3, 3, 3, 2, 1, 1, 1, 3, 3, 1, 0, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1, 2, 0, 2, 1, 1, 0, 0, 3, 0, 2, 0, 1, 3, 3, 2, 2, 3, 2, 1, 3, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 3, 2, 1, 3, 2, 2, 3, 3, 2, 2, 3, 3, 2, 1, 1, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 2, 3, 3, 1, 2, 3, 1, 3, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 3, 1, 1, 2, 2, 1, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 3, 3, 2, 2, 2, 2, 0, 1, 0, 3, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
2,0,2003
1,1,1536
3,2,1474
0,3,1192
4,4,31


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('دن', 0.03986029101081207), ('بیشک', 0.03578423386032328), ('سو', 0.03069760500074955), ('عذاب', 0.02987676650186663), ('کوئی', 0.028229043483515938), ('لوگ', 0.02803897644008738), ('جائیں', 0.027007769750185824), ('تمہیں', 0.024341894372378162), ('بے', 0.024181450327993095), ('لوگوں', 0.02210631701234625)], 1: [('السلام', 0.05964421634434929), ('اﷲ', 0.05836673033708564), ('علیہ', 0.05353738108272417), ('رب', 0.04749247173182729), ('والا', 0.04087564464633481), ('بیشک', 0.04026168682742313), ('اے', 0.0332387760968113), ('موسی', 0.033026927184769204), ('فرمایا', 0.03274930361762208), ('زمین', 0.03218301783000871)], 2: [('اﷲ', 0.0545139308784437), ('لوگ', 0.051746999754924965), ('رب', 0.045537315080923665), ('لوگوں', 0.04130173028921887), ('ایمان', 0.039968990969628006), ('بیشک', 0.0397620180928464), ('کوئی', 0.03717594366775971), ('عذاب', 0.037017099792382505), ('دوزخ', 0.02825953770800021), ('تمہارے', 0.02625436711026594)], 3: [('اللہ', 0.13392378592771), ('علیہ', 0.039836797846

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2003,0_دن_بیشک_سو_عذاب,"[دن, بیشک, سو, عذاب, کوئی, لوگ, جائیں, تمہیں, ...",[انہوں نے کہا: اے میری قوم! بے شک میں تمہیں وا...
1,1,1536,1_السلام_اﷲ_علیہ_رب,"[السلام, اﷲ, علیہ, رب, والا, بیشک, اے, موسی, ف...",[(فرشتوں نے) کہا: ایسے ہی ہوگا، تمہارے رب نے ف...
2,2,1474,2_اﷲ_لوگ_رب_لوگوں,"[اﷲ, لوگ, رب, لوگوں, ایمان, بیشک, کوئی, عذاب, ...",[بیشک جو لوگ ایمان لائے اور انہوں نے نیک اعمال...
3,3,1192,3_اللہ_علیہ_صلی_وسلم,"[اللہ, علیہ, صلی, وسلم, وآلہ, والا, رسول, ایما...",[اور تمہیں کیا ہوگیا ہے کہ تم اللہ پر ایمان نہ...
4,4,31,4_کن_جھٹلاؤ_نعمتوں_دونوں,"[کن, جھٹلاؤ, نعمتوں, دونوں, پس, رب, گروہ, انسا...",[پس تم دونوں اپنے رب کی کن کن نعمتوں کو جھٹلاؤ...


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(data, window=3, min_similarity=0.01)

100%|██████████| 7/7 [00:03<00:00,  1.79it/s]


In [ ]:
print(topic_distr)

[[0.14972434 0.22467191 0.27890451 0.34669924 0.        ]
 [0.08255424 0.26203486 0.16108788 0.49432301 0.        ]
 [0.14732899 0.35687771 0.21912798 0.27666531 0.        ]
 ...
 [0.1267817  0.34698609 0.34064902 0.18558319 0.        ]
 [0.19471967 0.17887713 0.38312833 0.24327487 0.        ]
 [0.14454182 0.         0.85545818 0.         0.        ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distribution")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
topic_model.update_topics(data, vectorizer_model=vectorizer_model)

In [ ]:
texts = [[word for word in str(document).split() if word not in stop_words_list] for document in data] #if word not in stop_words_list
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['دن', 'بیشک', 'سو', 'عذاب', 'کوئی', 'لوگ', 'جائیں', 'تمہیں', 'بے', 'لوگوں'], ['السلام', 'اﷲ', 'علیہ', 'رب', 'والا', 'بیشک', 'اے', 'موسی', 'فرمایا', 'زمین'], ['اﷲ', 'لوگ', 'رب', 'لوگوں', 'ایمان', 'بیشک', 'کوئی', 'عذاب', 'دوزخ', 'تمہارے'], ['اللہ', 'علیہ', 'صلی', 'وسلم', 'وآلہ', 'والا', 'رسول', 'ایمان', 'بیشک', 'لوگ'], ['کن', 'جھٹلاؤ', 'نعمتوں', 'دونوں', 'پس', 'رب', 'گروہ', 'انسان', 'اے', '']]


In [ ]:
# compute Coherence Score CV

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence)


CV Score:  0.53


In [ ]:
# compute Coherence Score NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_npmi')
coherence = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence)


NPMI Score:  0.04


**Diversity Score**

upload rbo.py file before importing

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.87


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2024-06-09 09:34:52,892 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2003,0_دن_بیشک_سو_عذاب,"[دن, بیشک, سو, عذاب, کوئی, لوگ, جائیں, تمہیں, ...",[انہوں نے کہا: اے میری قوم! بے شک میں تمہیں وا...
1,1,1536,1_السلام_اﷲ_علیہ_رب,"[السلام, اﷲ, علیہ, رب, والا, بیشک, اے, موسی, ف...",[(فرشتوں نے) کہا: ایسے ہی ہوگا، تمہارے رب نے ف...
2,2,1474,2_اﷲ_لوگ_رب_لوگوں,"[اﷲ, لوگ, رب, لوگوں, ایمان, بیشک, کوئی, عذاب, ...",[بیشک جو لوگ ایمان لائے اور انہوں نے نیک اعمال...
3,3,1192,3_اللہ_علیہ_صلی_وسلم,"[اللہ, علیہ, صلی, وسلم, وآلہ, والا, رسول, ایما...",[اور تمہیں کیا ہوگیا ہے کہ تم اللہ پر ایمان نہ...
4,4,31,4_کن_جھٹلاؤ_نعمتوں_دونوں,"[کن, جھٹلاؤ, نعمتوں, دونوں, پس, رب, گروہ, انسا...",[پس تم دونوں اپنے رب کی کن کن نعمتوں کو جھٹلاؤ...


# LDA

We use the [ parallelized Latent Dirichlet Allocation (LDA)](https://radimrehurek.com/gensim/models/ldamulticore.html) from Gensim.

Note: for LDA you have to define topics number in advance.

In [ ]:
n_topics=5
lda = LdaMulticore(corpus, id2word=id2word, random_state=42, num_topics=5)
topics = lda.show_topics(num_topics=5, formatted=False)

#Extract the words from the topics
topics_list = []
for _, topic_words in topics:
    words = [word for word, _ in topic_words]
    topics_list.append(words)

print(topics_list)

In [ ]:
#CV Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_lda)


CV Score:  0.44


In [ ]:
#NPMI Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_lda = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_lda)


NPMI Score:  -0.02


In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_LDA = round(inverted_rbo_calculator.irbo(topics_list, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_LDA)


IRBO Score:  0.36


#NMF
We use Gensim library for implementation of NMF

Note: for NMF you have to define topics number in advance.

In [ ]:
#Using Gensim
from gensim import corpora, models


# Train NMF model
num_topics = 5  # Define the number of topics
nmf_model = models.Nmf(corpus, num_topics=num_topics, id2word=id2word, random_state=42)

# Extract topics
topics = []
for topic_id in range(num_topics):
    topic_words = nmf_model.show_topic(topic_id, topn=10)
    topic_words = [word for word, _ in topic_words]
    topics.append(topic_words)

# Print topics in the desired format
print(topics)

In [ ]:
#Calculate CV Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_nmf = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_nmf)

In [ ]:
#Calculate NPMI Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_nmf = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_nmf)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_NMF = round(inverted_rbo_calculator.irbo(topics, topk=10, weight=0.9),2)c
print('\nIRBO Score: ',IRBO_NMF)